In [ ]:
import google.generativeai as genai
import pandas as pd
import time

# Set API key directly
genai.configure(api_key="API_KEY")

In [ ]:
#  Call Gemini API for Text-to-SQL generation
def get_sql_query_from_gemini(natural_language_input, attempt=1):
    model = genai.GenerativeModel("gemini-1.5-flash")

    # Detailed prompt for generating SQL
    prompt = f"""
    Translate the following natural language query to an optimized and correct SQL query. Ensure the SQL handles any necessary joins, filtering conditions, sorting, or aggregations correctly.

    Natural Language Query: {natural_language_input}
    """

    # Generate  SQL query using the Gemini API
    response = model.generate_content(prompt)

    # Extract SQL query from the response text (removing explanations)
    try:
        sql_query = response.text.split('```sql')[1].split('```')[0].strip()
    except IndexError:
        if attempt <= 3:
            print(f"Retrying SQL generation for query: {natural_language_input} (Attempt {attempt})")
            return get_sql_query_from_gemini(natural_language_input, attempt + 1)
        else:
            sql_query = "Failed to generate SQL"

    return sql_query


In [ ]:
#  Evaluate generated SQL using Gemini
def evaluate_generated_sql_gemini(natural_language_query, generated_sql):
    model = genai.GenerativeModel("gemini-1.5-flash")

    # Prepare the evaluation prompt with instructions for feedback
    prompt = f"""
    Here is the natural language query: "{natural_language_query}"
    Here is the generated SQL query: "{generated_sql}"
    Is this SQL query:
    - Fully correct
    - Partially correct (with minor syntactical differences)
    - Structurally correct (correct structure but missing minor components)
    - Incorrect (completely wrong or produces incorrect results)
    If incorrect or partially correct, suggest improvements.
    """

    #  Evaluation response from the Gemini model
    response = model.generate_content(prompt)
    evaluation = response.text.strip()

    # Assign scores based on the evaluation response
    if 'fully correct' in evaluation.lower():
        return 1.0, evaluation
    elif 'partially correct' in evaluation.lower():
        return 0.8, evaluation
    elif 'structurally correct' in evaluation.lower():
        return 0.5, evaluation
    else:
        return 0.0, evaluation


In [ ]:
#  Generate and evaluate multiple queries
def generate_and_evaluate_queries(test_queries, delay=10):
    generated_queries = []
    evaluation_scores = []
    evaluation_feedback = []

    for idx, row in test_queries.iterrows():
        natural_language_query = row['Natural Language Query']

        # Generate SQL query using the Gemini API
        generated_sql = get_sql_query_from_gemini(natural_language_query)
        generated_queries.append(generated_sql)

        # Evaluate the generated SQL query
        score, feedback = evaluate_generated_sql_gemini(natural_language_query, generated_sql)
        evaluation_scores.append(score)
        evaluation_feedback.append(feedback)

        print(f"Query {idx + 1}: {natural_language_query}")
        print(f"Generated SQL: {generated_sql}")
        print(f"Score: {score * 100}%\n")

        # Add a delay between requests to avoid quota exhaustion
        time.sleep(delay)

    test_queries['Generated_SQL'] = generated_queries
    test_queries['Evaluation_Score'] = evaluation_scores
    test_queries['Evaluation_Feedback'] = evaluation_feedback

    return test_queries


In [ ]:
# Load  test queries
test_queries = pd.read_excel(r"E:\Pagila Evals Dataset.xlsx")

# Generate and evaluate  queries with  delay between requests
test_queries_with_results = generate_and_evaluate_queries(test_queries, delay=10)

# Save  results to an Excel file
test_queries_with_results.to_excel(r"E:\Pagila_Evals_Dataset_Results.xlsx", index=False)
print(f"Results saved to Excel file.")